In [7]:
!pip install speechbrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.1/760.1 kB 13.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

In [8]:
%%capture
!pip install datasets -U
!pip install librosa
!pip install jiwer

In [9]:
from speechbrain.inference.speaker import EncoderClassifier
import torch
from collections import defaultdict
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.transforms import Compose
from transformers import WhisperForConditionalGeneration, WhisperProcessor, AutoFeatureExtractor, WhisperModel
from datasets import load_dataset, DatasetDict, Audio

In [10]:
common_voice = DatasetDict()

common_voice_train = load_dataset("fsicoli/common_voice_17_0", "ja", split="train")
common_voice_test = load_dataset("fsicoli/common_voice_17_0", "ja", split="test")

print(common_voice)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for fsicoli/common_voice_17_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/fsicoli/common_voice_17_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `d

Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 4156it [00:00, 41552.91it/s]
Reading metadata...: 10039it [00:00, 47972.61it/s]


Generating validation split: 0 examples [00:00, ? examples/s]


Reading metadata...: 6261it [00:00, 74176.44it/s]


...

Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 8831it [00:00, 88297.87it/s]
Reading metadata...: 17737it [00:00, 88742.47it/s]
Reading metadata...: 27868it [00:00, 94477.82it/s]
Reading metadata...: 37753it [00:00, 96184.94it/s]
Reading metadata...: 47372it [00:00, 95103.41it/s]
Reading metadata...: 56885it [00:00, 94445.38it/s]
Reading metadata...: 66403it [00:00, 94680.14it/s]
Reading metadata...: 75873it [00:00, 93142.94it/s]
Reading metadata...: 85193it [00:00, 89187.18it/s]
Reading metadata...: 94145it [00:01, 88443.16it/s]
Reading metadata...: 103522it [00:01, 90014.59it/s]
Reading metadata...: 112544it [00:01, 86797.23it/s]
Reading metadata...: 122075it [00:01, 89264.04it/s]
Reading metadata...: 131992it [00:01, 92156.90it/s]
Reading metadata...: 141744it [00:01, 93736.27it/s]
Reading metadata...: 151805it [00:01, 95768.86it/s]
Reading metadata...: 161404it [00:01, 95486.82it/s]
Reading metadata...: 170968it [00:01, 94513.74it/s]
Reading metadata...: 180432it [00:

Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 13547it [00:00, 89732.79it/s]


DatasetDict({
    
})


In [11]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "down_votes", "gender", "locale", "segment", "up_votes"])

In [12]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [26]:
## ECAPA encoding
classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb")
signal = common_voice_train[0]["audio"]["array"]
# fs = common_voice_train[0]["audio"]["sampling_rate"]
embeddings = classifier.encode_batch(torch.tensor(signal))

## Whisper Encoding
model = WhisperModel.from_pretrained("openai/whisper-base")
feature_extractor = AutoFeatureExtractor.from_pretrained("openai/whisper-base")

inputs = feature_extractor(common_voice_train[0]["audio"]["array"], sampling_rate=common_voice_train[0]["audio"]["sampling_rate"], return_tensors="pt")
input_features = inputs.input_features
decoder_input_ids = torch.tensor([[1, 1]]) * model.config.decoder_start_token_id
last_hidden_state = model(input_features, decoder_input_ids=decoder_input_ids).encoder_last_hidden_state
list(last_hidden_state.shape)

[1, 1500, 512]

In [14]:
embeddings.shape

torch.Size([1, 1, 192])

In [28]:
last_hidden_state = model(input_features, decoder_input_ids=decoder_input_ids)

In [31]:
model

WhisperModel(
  (encoder): WhisperEncoder(
    (conv1): Conv1d(80, 512, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(512, 512, kernel_size=(3,), stride=(2,), padding=(1,))
    (embed_positions): Embedding(1500, 512)
    (layers): ModuleList(
      (0-5): 6 x WhisperEncoderLayer(
        (self_attn): WhisperSdpaAttention(
          (k_proj): Linear(in_features=512, out_features=512, bias=False)
          (v_proj): Linear(in_features=512, out_features=512, bias=True)
          (q_proj): Linear(in_features=512, out_features=512, bias=True)
          (out_proj): Linear(in_features=512, out_features=512, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (activation_fn): GELUActivation()
        (fc1): Linear(in_features=512, out_features=2048, bias=True)
        (fc2): Linear(in_features=2048, out_features=512, bias=True)
        (final_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    

Embedding shape from Whisper: Batch X 1500 X 512 --> Batch X 1500 X 64

Embedding shape from ECAPA: Batch X 1 X 192 --> Batch X 1 X 64

In [ ]:
# Define your downsampling convolutional layer
downsample_conv = nn.Conv1d(in_channels=512, out_channels=64, kernel_size=1)

# Assuming your input tensor is named input_tensor
# Reshape the tensor to fit the convolutional layer
input_tensor = torch.ones(2,1500,512)
input_tensor = input_tensor.permute(0, 2, 1)  # Change shape to batch * 512 * 1500

# Apply convolution
output_tensor = downsample_conv(input_tensor)

# Check the shape
print(output_tensor.shape)  # Should be batch * 64 * 500


torch.Size([2, 64, 1500])


In [18]:
x = torch.ones(2, 1500, 512)
x = torch.mean(x,dim=1)
print(x.shape)

torch.Size([2, 512])


In [25]:
class SpeechModel(torch.nn.Module):
    def __init__(self):
        super(SpeechModel, self).__init__()
        self.sampling_rate = 16000
        self.num_speaker_class = 10
        self.whisper_encoder = WhisperModel.from_pretrained("openai/whisper-base")
        self.decoder_input_ids = torch.tensor([[1, 1]]) * self.whisper_encoder.config.decoder_start_token_id
        self.whisper_feature_extractor = AutoFeatureExtractor.from_pretrained("openai/whisper-base")
        self.ecapa_encoder = EncoderClassifier.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb")

        # Define downsampling layers
        self.whisper_downsample = nn.Conv1d(in_channels=512, out_channels=64, kernel_size=1)
        self.ecapa_downsample = nn.Conv1d(in_channels=192, out_channels=64, kernel_size=1)

        # Define Transformer layers
        self.transformer_encoder_single = nn.TransformerEncoderLayer(d_model=128, nhead=8, dim_feedforward=512, batch_first=True)
        self.transformer_decoder_single = nn.TransformerDecoderLayer(d_model=128, nhead=8, dim_feedforward=512, batch_first=True)
        self.transformer_encoder_stack = nn.TransformerEncoder(self.transformer_encoder_single, num_layers=6)
        self.transformer_decoder_stack = nn.TransformerDecoder(self.transformer_decoder_single, num_layers=6)

        # Define prediction heads
        #self.next_word_prediction_head = nn.Linear(...)
        self.speaker_recognition_head = nn.Linear(128,self.num_speaker_class)
        self.speaker_softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        # Forward pass for whisper branch
        x_whisper = self.whisper_feature_extractor(x, sampling_rate=self.sampling_rate, return_tensors="pt").input_features
        whisper_embedding = self.whisper_encoder(x_whisper, decoder_input_ids=self.decoder_input_ids).encoder_last_hidden_state
        whisper_embedding = whisper_embedding.permute(0, 2, 1)
        downsampled_whisper_embedding = self.whisper_downsample(whisper_embedding)
        downsampled_whisper_embedding = downsampled_whisper_embedding.permute(0, 2, 1)
        downsampled_whisper_embedding = torch.mean(downsampled_whisper_embedding, dim=1)

        # Forward pass for ECAPA branch
        ecapa_embedding = self.ecapa_encoder.encode_batch(x)
        ecapa_embedding = ecapa_embedding.permute(0, 2, 1)
        downsampled_ecapa_embedding = self.ecapa_downsample(ecapa_embedding)
        downsampled_ecapa_embedding = downsampled_ecapa_embedding.permute(0, 2, 1)
        downsampled_ecapa_embedding = torch.squeeze(downsampled_ecapa_embedding,dim=1)

        # Concatenate downscaled embeddings
        concatenated_embeddings = torch.cat((downsampled_whisper_embedding, downsampled_ecapa_embedding), dim=-1)

        # Transformer layers
        transformer_output = self.transformer_encoder_stack(concatenated_embeddings)
        transformer_decoder_output = self.transformer_decoder_stack(concatenated_embeddings, transformer_output)

        # Task-specific heads
        word_prediction = transformer_decoder_output
        speaker_recognition = self.speaker_recognition_head(transformer_output)
        speaker_recognition = self.speaker_softmax(speaker_recognition)

        #next_word_prediction = self.next_word_prediction_head(transformer_output

        return transformer_decoder_output #,next_word_prediction, speaker_recognition

# Define model
model = SpeechModel()
input_data = torch.tensor(common_voice_train[0]["audio"]["array"])
output = model.forward(input_data)
print(output.shape)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


torch.Size([1, 128])


In [ ]:
# Define loss functions
next_word_loss_function = nn.CrossEntropyLoss()
speaker_recognition_loss_function = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define data preprocessing
transform = Compose([preprocess_function])

# Load dataset
dataset = CommonVoiceDataset(root_dir='path_to_commonvoice_dataset', transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for batch in dataloader:
        optimizer.zero_grad()
        inputs, targets = batch
        next_word_prediction, speaker_recognition = model(inputs)

        # Calculate loss
        next_word_loss = next_word_loss_function(next_word_prediction, targets)
        # Calculate speaker recognition loss
        speaker_recognition_loss = speaker_recognition_loss_function(speaker_recognition, speaker_labels)

        total_loss = next_word_loss + speaker_recognition_loss

        # Backpropagation
        total_loss.backward()
        optimizer.step()